In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# парамс
DATASET_PATH = "data"
IMG_SIZE = (64, 64)
NUM_SAMPLES = 400  

In [ ]:
def load_images(folder, label, num_samples):
    images = []
    labels = []
    files = os.listdir(folder)[:num_samples]
    for file in tqdm(files, desc=f"Loading {label}"):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, IMG_SIZE)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
            images.append(img.flatten())
            labels.append(label)
    return images, labels

cats, cat_labels = load_images(os.path.join(DATASET_PATH, "cat"), 0, NUM_SAMPLES)
dogs, dog_labels = load_images(os.path.join(DATASET_PATH, "dog"), 1, NUM_SAMPLES)

FileNotFoundError: [Errno 2] No such file or directory: 'data/cat'

In [ ]:
X = np.array(cats + dogs)
y = np.array(cat_labels + dog_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# прост классификатор
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_no_pca = accuracy_score(y_test, y_pred)
print("Accuracy without PCA:", accuracy_no_pca)